In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

import cv2
import glob
import torch
import pickle
import keyboard
import datetime
import numpy as np
from PIL import Image
from utils.es_utils import *
from utils.env_utils import *
from utils.eval_utils import *
from utils.train_utils import *
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from models.Buffer import Buffer, Pool
from marl_aquarium import aquarium_v0
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

In [2]:
'''
Trajectory Prediction under Survival Pressure (Liang):
    - Vorhersage von Fisch-Trajektorien mit den drei States "ohne Predator", "mit Predator", "Angriff Predator" (Monte-Carlo simulieren von bspw. 25-frames Traj durch Model mit realem Startzustand)
    - Descriptive Beschreibung wie sich Schwarm durch Präsenz des Predators ändert (Polarization, Angular Momentum, Density, ...)
    - Korrelationsberechnungen zwischen Schwarmmetriken und bspw. Predator-Distanz - Distanz zwischen Fischen nutzen wie im Parisi Paper
'''

'\nTrajectory Prediction under Survival Pressure (Liang):\n    - Vorhersage von Fisch-Trajektorien mit den drei States "ohne Predator", "mit Predator", "Angriff Predator" (Monte-Carlo simulieren von bspw. 25-frames Traj durch Model mit realem Startzustand)\n    - Descriptive Beschreibung wie sich Schwarm durch Präsenz des Predators ändert (Polarization, Angular Momentum, Density, ...)\n    - Korrelationsberechnungen zwischen Schwarmmetriken und bspw. Predator-Distanz - Distanz zwischen Fischen nutzen wie im Parisi Paper\n'

## Prey-Only Prediction

In [6]:
# GAIL Model Path
gail_folder = "GAIL Training - 19.11.2025_18.52 - Couzin Data"
model_folder = rf"..\data\2. Training\training"
gail_path = os.path.join(model_folder, "GAIL", gail_folder)

gail_prey_only_policy = torch.load(os.path.join(gail_path, "gail_prey_policy.pt"), weights_only=False)
gail_pred_only_policy = torch.load(os.path.join(gail_path, "gail_pred_policy.pt"), weights_only=False)

# Startframe pool
ftw_path = rf'..\data\1. Data Processing\processed\video\3. full_track_windows'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

AttributeError: Can't get attribute 'PredatorAttention' on <module 'models.ModularNetworks' from 'c:\\Users\\janni\\OneDrive\\Dokumente\\Privat\\Bildung\\M. Sc. Social and Economic Data Science\\4. Semester\\Master Thesis\\Code\\notebooks\\models\\ModularNetworks.py'>

In [4]:
start_frame_pool = Pool(max_length=12100, device=device)
start_frame_pool.generate_startframes(ftw_path)
print(f"Start frames in pool: {len(start_frame_pool)}")

env = parallel_env(predator_count=0, prey_count=32, action_count=360, use_walls=True)
positions = start_frame_pool.sample(n=1)
env.reset(options=positions)
print("Environment initialized")

Start frames in pool: 11978
Environment initialized


In [5]:
env = parallel_env(predator_count=0, prey_count=32, action_count=360, use_walls=False)
positions = start_frame_pool.sample(n=1)
env.reset(options=positions)
metrics = run_policies(env, gail_pred_only_policy, gail_prey_only_policy, prey_features=5)

NameError: name 'gail_prey_only_policy' is not defined